# Part I Training LDA model (50 pts)

Use the bodies of articles in your Webhose dataset to train an LDA model.  Your program should output topic clusters and up to 10 keywords in each cluster. Clusters should not overlap, and keywords should allow to approximate the meaning.

### preperation

#### read data

In [1]:
import webhoseio, os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import nltk, re, spacy
from nltk.stem.wordnet import WordNetLemmatizer
import pyLDAvis
from pyLDAvis import sklearn

stopwords = set(nltk.corpus.stopwords.words('english'))

In [2]:
#read data
import json
json_data = open("/Users/arlenehuang/OneDrive - Columbia University/Summer 2020/APAN 5430/Assignment 7/webhose_Citigroup_deduplicated.json").readlines()
newsfeeds = []

for line in json_data:
    newsfeeds.append(json.loads(line))

In [3]:
#get text
feeds = []
for line in newsfeeds:
    feeds.append(line['text'])

#### define useful functions and prepare data

In [4]:
#Run in terminal: python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

In [5]:
#tokenize, clean-up and lemmatize

def tokenize(text):
    import gensim  #just to use simple_preprocess
    tokens = gensim.utils.simple_preprocess(text, deacc=True)
    lmtzr = WordNetLemmatizer()
    filtered_tokens = []
    for token in tokens:
        token = token.replace("'s", " ").replace("n’t", " not").replace("’ve", " have")
        token = re.sub(r'[^a-zA-Z0-9 ]', '', token)
        if token not in stopwords:
            filtered_tokens.append(token.lower())
    return filtered_tokens

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

def tokenize_lemmatize(text):
    data_words = []
    for line in text:
        data_words.append(tokenize(line))
    #Run in terminal: python3 -m spacy download en
    nlp = spacy.load('en', disable=['parser', 'ner']) #just incase
    data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    return data_lemmatized

In [6]:
data_lemmatized = tokenize_lemmatize(feeds)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


### modify parameters

#### modify max_iter

To modify max_iter, I will try max_iter = [5,10,50]. For now, I will start with max_features = 100 and min_df = 0.1, max_df = 0.9.

when max_iter = 5

In [13]:
n_top_words = 10
vectorizer = CountVectorizer(min_df = 0.1, max_df = 0.9,
                             max_features=100,
                             ngram_range=(2,4))
data_vectorized = vectorizer.fit_transform(data_lemmatized)
data_dense = data_vectorized.todense()
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")
lda_model = LatentDirichletAllocation(n_components=10,
                                      max_iter=5,
                                      learning_method='online',   
                                      random_state=1,
                                      learning_offset=10.)
lda_output = lda_model.fit_transform(data_vectorized)
tf_feature_names = vectorizer.get_feature_names()
topics = dict()
for topic_idx, topic in enumerate(lda_model.components_):
    topics[topic_idx] = [tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
    print("Topic #%d:" % topic_idx)
    print(" | ".join([tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
print("Log Likelihood: ", lda_model.score(data_vectorized))
print("Perplexity: ", lda_model.perplexity(data_vectorized))
pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)
vis

Sparsicity:  49.55900996076064 %
Topic #0:
company stock | share company | share company stock | own share company | own share company stock | own share | company stock value | share company stock value | stock value | company stock worth
Topic #1:
research note | rating research note | rating research | hold rating | sell rating | move average | price target | neutral rating | buy rating | rating hold
Topic #2:
purchase additional | purchase additional share | purchase additional share last | share last | last quarter | additional share last | share last quarter | additional share last quarter | own share | additional share
Topic #3:
buy rating | average price | day move | day move average | move average | price target | hold rating | target price | move average price | day move average price
Topic #4:
purchase additional | purchase additional share | own share | share last | share last quarter | additional share last | additional share last quarter | last quarter | purchase additiona

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.130778  0.040510       1        1  19.408457
7     -0.131420  0.024656       2        1  15.472144
3      0.183775  0.048786       3        1  14.573961
8      0.069726  0.048738       4        1  10.944121
2     -0.127870  0.007803       5        1   9.759574
6     -0.029415  0.042940       6        1   9.309433
5      0.165328 -0.151110       7        1   8.944621
4     -0.103686 -0.124365       8        1   5.893390
1      0.118048  0.086320       9        1   5.588494
9     -0.013709 -0.024277      10        1   0.105805, topic_info=                             Term         Freq        Total Category  logprob  \
80                  research note  4776.000000  4776.000000  Default  30.0000   
91             share last quarter  4456.000000  4456.000000  Default  29.0000   
3           additional share last  4456.000000  4456.000000  Default  28.0000   
4   additional share last quarter  4446.000000  4446.000000  Default  27.0000   
69           rating research note  2069.000000  2069.000000  Default  26.0000   
..                            ...          ...          ...      ...      ...   
98                    stock worth     3.073795  3691.981403  Topic10  -4.6047   
64                 rating company     3.073641  2765.623898  Topic10  -4.6048   
68                rating research     3.073373  3886.907878  Topic10  -4.6049   
39                   move average     3.073298  5898.453579  Topic10  -4.6049   
87                  share company     3.073123  4923.443067  Topic10  -4.6049   

    loglift  
80  30.0000  
91  29.0000  
3   28.0000  
4   27.0000  
69  26.0000  
..      ...  
98  -0.2397  
64   0.0492  
68  -0.2913  
39  -0.7084  
87  -0.5277  

[423 rows x 6 columns], token_table=      Topic      Freq                Term
term                                     
0         1  0.311798  acquire additional
0         2  0.535278  acquire additional
0         4  0.007319  acquire additional
0         6  0.126866  acquire additional
0         7  0.002440  acquire additional
...     ...       ...                 ...
99        6  0.055155        target price
99        7  0.105582        target price
99        8  0.041169        target price
99        9  0.049048        target price
99       10  0.000591        target price

[917 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 8, 4, 9, 3, 7, 6, 5, 2, 10])

when max_iter = 10

In [14]:
n_top_words = 10
vectorizer = CountVectorizer(min_df = 0.1, max_df = 0.9,
                             max_features=100,
                             ngram_range=(2,4))
data_vectorized = vectorizer.fit_transform(data_lemmatized)
data_dense = data_vectorized.todense()
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")
lda_model = LatentDirichletAllocation(n_components=10,
                                      max_iter=10,
                                      learning_method='online',   
                                      random_state=1,
                                      learning_offset=10.)
lda_output = lda_model.fit_transform(data_vectorized)
tf_feature_names = vectorizer.get_feature_names()
topics = dict()
for topic_idx, topic in enumerate(lda_model.components_):
    topics[topic_idx] = [tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
    print("Topic #%d:" % topic_idx)
    print(" | ".join([tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
print("Log Likelihood: ", lda_model.score(data_vectorized))
print("Perplexity: ", lda_model.perplexity(data_vectorized))
pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)
vis

Sparsicity:  49.55900996076064 %
Topic #0:
company stock | share company | share company stock | own share company | own share company stock | own share | company stock value | share company stock value | stock value | company stock worth
Topic #1:
research note | rating research note | rating research | hold rating | sell rating | move average | rating hold | neutral rating | buy rating | price target
Topic #2:
purchase additional | purchase additional share | share last | additional share last | additional share last quarter | share last quarter | last quarter | purchase additional share last | own share | additional share
Topic #3:
buy rating | average price | day move | day move average | move average | move average price | day move average price | price target | hold rating | target price
Topic #4:
purchase additional | purchase additional share | share last | share last quarter | additional share last | additional share last quarter | last quarter | own share | purchase additiona

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.136834 -0.074406       1        1  18.919016
3     -0.183837 -0.012634       2        1  15.096188
7      0.159140 -0.010448       3        1  14.470602
8     -0.068603 -0.036230       4        1  11.296580
6      0.046255 -0.040851       5        1   9.847935
2      0.129253  0.022533       6        1   9.566177
5     -0.183504  0.142329       7        1   9.386947
4      0.099137  0.121583       8        1   5.881769
1     -0.151291 -0.130645       9        1   5.428980
9      0.016617  0.018769      10        1   0.105805, topic_info=                    Term         Freq        Total Category  logprob  loglift
80         research note  4735.000000  4735.000000  Default  30.0000  30.0000
88   share company stock  3768.000000  3768.000000  Default  29.0000  29.0000
69  rating research note  2051.000000  2051.000000  Default  28.0000  28.0000
91    share last quarter  4486.000000  4486.000000  Default  27.0000  27.0000
90            share last  4560.000000  4560.000000  Default  26.0000  26.0000
..                   ...          ...          ...      ...      ...      ...
64        rating company     3.072484  2761.705672  Topic10  -4.6052   0.0502
68       rating research     3.072473  3866.496347  Topic10  -4.6052  -0.2863
39          move average     3.072471  5890.448921  Topic10  -4.6052  -0.7073
87         share company     3.072463  4917.995642  Topic10  -4.6052  -0.5269
86            sell share     3.072459  2160.434444  Topic10  -4.6052   0.2957

[424 rows x 6 columns], token_table=      Topic      Freq                Term
term                                     
0         1  0.266696  acquire additional
0         3  0.550880  acquire additional
0         4  0.003400  acquire additional
0         5  0.171968  acquire additional
0         7  0.000486  acquire additional
...     ...       ...                 ...
99        6  0.050458        target price
99        7  0.123780        target price
99        8  0.041786        target price
99        9  0.042771        target price
99       10  0.000591        target price

[866 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 8, 9, 7, 3, 6, 5, 2, 10])

when max_iter = 50

In [18]:
n_top_words = 10
vectorizer = CountVectorizer(min_df = 0.1, max_df = 0.9,
                             max_features=100,
                             ngram_range=(2,4))
data_vectorized = vectorizer.fit_transform(data_lemmatized)
data_dense = data_vectorized.todense()
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")
lda_model = LatentDirichletAllocation(n_components=10,
                                      max_iter=50,
                                      learning_method='online',   
                                      random_state=1,
                                      learning_offset=10.)
lda_output = lda_model.fit_transform(data_vectorized)
tf_feature_names = vectorizer.get_feature_names()
topics = dict()
for topic_idx, topic in enumerate(lda_model.components_):
    topics[topic_idx] = [tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
    print("Topic #%d:" % topic_idx)
    print(" | ".join([tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
print("Log Likelihood: ", lda_model.score(data_vectorized))
print("Perplexity: ", lda_model.perplexity(data_vectorized))
pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)
vis

Sparsicity:  49.55900996076064 %
Topic #0:
company stock | share company | share company stock | own share company | own share company stock | company stock value | own share | share company stock value | stock value | company stock worth
Topic #1:
research note | rating research note | rating research | hold rating | sell rating | rating hold | move average | neutral rating | buy rating | rating buy
Topic #2:
purchase additional | purchase additional share | share last | additional share last | additional share last quarter | share last quarter | last quarter | purchase additional share last | additional share | own share
Topic #3:
buy rating | average price | day move | day move average | move average | move average price | day move average price | price target | hold rating | target price
Topic #4:
purchase additional | purchase additional share | share last | share last quarter | additional share last | additional share last quarter | last quarter | purchase additional share last |

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.142024 -0.209052       1        1  17.704970
3     -0.185216 -0.000379       2        1  16.546015
7      0.208880  0.046550       3        1  13.404835
8     -0.063862 -0.008451       4        1  12.436169
5     -0.194079  0.082215       5        1  10.148751
6      0.084776 -0.020281       6        1   9.649671
2      0.130521  0.072339       7        1   9.609743
4      0.079196  0.128162       8        1   5.667990
1     -0.222931 -0.079327       9        1   4.726052
9      0.020692 -0.011776      10        1   0.105805, topic_info=                    Term         Freq        Total Category  logprob  loglift
80         research note  4736.000000  4736.000000  Default  30.0000  30.0000
88   share company stock  3773.000000  3773.000000  Default  29.0000  29.0000
45     own share company  3627.000000  3627.000000  Default  28.0000  28.0000
87         share company  4925.000000  4925.000000  Default  27.0000  27.0000
69  rating research note  2058.000000  2058.000000  Default  26.0000  26.0000
..                   ...          ...          ...      ...      ...      ...
65           rating hold     3.072437  2363.802288  Topic10  -4.6052   0.2058
80         research note     3.072437  4736.942908  Topic10  -4.6052  -0.4894
64        rating company     3.072437  2750.041844  Topic10  -4.6052   0.0544
59           quick ratio     3.072437  2473.282057  Topic10  -4.6052   0.1605
85           sell rating     3.072437  3981.365993  Topic10  -4.6052  -0.3156

[420 rows x 6 columns], token_table=      Topic      Freq                      Term
term                                           
0         1  0.170919        acquire additional
0         3  0.565867        acquire additional
0         6  0.261690        acquire additional
0        10  0.001448        acquire additional
1         1  0.170426  acquire additional share
...     ...       ...                       ...
99        6  0.065488              target price
99        7  0.046495              target price
99        8  0.047286              target price
99        9  0.025325              target price
99       10  0.000594              target price

[774 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 8, 9, 6, 7, 3, 5, 2, 10])

From the results shown above, it seems that max_iter = 10 or 50 works the best.
For simplicity, I will take max_iter = 10 for following analysis

#### modify max_features

To modify max_features, I will try max_features = [50,100,500,5000]

when max_features = 50

In [23]:
n_top_words = 10
vectorizer = CountVectorizer(min_df = 0.1, max_df = 0.9,
                             max_features=50,
                             ngram_range=(2,4))
data_vectorized = vectorizer.fit_transform(data_lemmatized)
lda_model = LatentDirichletAllocation(n_components=10,
                                      max_iter=10,
                                      learning_method='online',   
                                      random_state=1,
                                      learning_offset=10.)
lda_output = lda_model.fit_transform(data_vectorized)
tf_feature_names = vectorizer.get_feature_names()
topics = dict()
for topic_idx, topic in enumerate(lda_model.components_):
    topics[topic_idx] = [tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
    print("Topic #%d:" % topic_idx)
    print(" | ".join([tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
print("Log Likelihood: ", lda_model.score(data_vectorized))
print("Perplexity: ", lda_model.perplexity(data_vectorized))
pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)
vis

Topic #0:
research note | rating research | hold rating | sell rating | move average | buy rating | price target | earning share | neutral rating | own share
Topic #1:
stock worth | own share | share last | last quarter | share last quarter | additional share last | additional share last quarter | additional share | price target | buy rating
Topic #2:
company stock | share company | share company stock | own share company | own share | average price | company stock value | own share company stock | stock value | buy rating
Topic #3:
stock value | own share | share last | last quarter | additional share last | share last quarter | additional share last quarter | additional share | buy rating | price target
Topic #4:
share company | company stock | share company stock | own share company | own share company stock | own share | company stock value | last quarter | share last | additional share last
Topic #5:
research report | rating research | sell rating | hold rating | move average | ne

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.178811  0.019880       1        1  16.126385
2      0.044090  0.173307       2        1  14.603081
1      0.102062 -0.055550       3        1  13.910579
3      0.111221 -0.031380       4        1  12.418197
4      0.180507  0.117131       5        1  11.507636
7      0.017493 -0.042878       6        1   7.939545
0     -0.111277 -0.024197       7        1   7.000818
9      0.141392 -0.144927       8        1   6.134563
5     -0.175703 -0.022092       9        1   5.705805
8     -0.130974  0.010706      10        1   4.653390, topic_info=                   Term         Freq        Total Category  logprob  loglift
38        research note  4679.000000  4679.000000  Default  30.0000  30.0000
39      research report  3521.000000  3521.000000  Default  29.0000  29.0000
49         target price  5120.000000  5120.000000  Default  28.0000  28.0000
36      rating research  3832.000000  3832.000000  Default  27.0000  27.0000
27  purchase additional  2601.000000  2601.000000  Default  26.0000  26.0000
..                  ...          ...          ...      ...      ...      ...
42        share company    54.882585  4915.801897  Topic10  -5.0980  -1.4274
0      additional share    73.116136  6754.150792  Topic10  -4.8111  -1.4583
5         average price    46.901291  4405.242699  Topic10  -5.2551  -1.4749
48          stock worth    31.242355  3756.773917  Topic10  -5.6614  -1.7220
11     day move average    16.985936  3775.549381  Topic10  -6.2708  -2.3363

[358 rows x 6 columns], token_table=      Topic      Freq              Term
term                                   
0         2  0.103936  additional share
0         3  0.237780  additional share
0         4  0.204023  additional share
0         5  0.103936  additional share
0         6  0.196620  additional share
...     ...       ...               ...
49        6  0.060351      target price
49        7  0.032226      target price
49        8  0.017383      target price
49        9  0.026758      target price
49       10  0.370505      target price

[451 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 3, 2, 4, 5, 8, 1, 10, 6, 9])

when max_features = 100

In [24]:
n_top_words = 10
vectorizer = CountVectorizer(min_df = 0.1, max_df = 0.9,
                             max_features=100,
                             ngram_range=(2,4))
data_vectorized = vectorizer.fit_transform(data_lemmatized)
lda_model = LatentDirichletAllocation(n_components=10,
                                      max_iter=10,
                                      learning_method='online',   
                                      random_state=1,
                                      learning_offset=10.)
lda_output = lda_model.fit_transform(data_vectorized)
tf_feature_names = vectorizer.get_feature_names()
topics = dict()
for topic_idx, topic in enumerate(lda_model.components_):
    topics[topic_idx] = [tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
    print("Topic #%d:" % topic_idx)
    print(" | ".join([tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
print("Log Likelihood: ", lda_model.score(data_vectorized))
print("Perplexity: ", lda_model.perplexity(data_vectorized))
pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)
vis

Topic #0:
company stock | share company | share company stock | own share company | own share company stock | own share | company stock value | share company stock value | stock value | company stock worth
Topic #1:
research note | rating research note | rating research | hold rating | sell rating | move average | rating hold | neutral rating | buy rating | price target
Topic #2:
purchase additional | purchase additional share | share last | additional share last | additional share last quarter | share last quarter | last quarter | purchase additional share last | own share | additional share
Topic #3:
buy rating | average price | day move | day move average | move average | move average price | day move average price | price target | hold rating | target price
Topic #4:
purchase additional | purchase additional share | share last | share last quarter | additional share last | additional share last quarter | last quarter | own share | purchase additional share last | additional share
T

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.136834 -0.074406       1        1  18.919016
3     -0.183837 -0.012634       2        1  15.096188
7      0.159140 -0.010448       3        1  14.470602
8     -0.068603 -0.036230       4        1  11.296580
6      0.046255 -0.040851       5        1   9.847935
2      0.129253  0.022533       6        1   9.566177
5     -0.183504  0.142329       7        1   9.386947
4      0.099137  0.121583       8        1   5.881769
1     -0.151291 -0.130645       9        1   5.428980
9      0.016617  0.018769      10        1   0.105805, topic_info=                    Term         Freq        Total Category  logprob  loglift
80         research note  4735.000000  4735.000000  Default  30.0000  30.0000
88   share company stock  3768.000000  3768.000000  Default  29.0000  29.0000
69  rating research note  2051.000000  2051.000000  Default  28.0000  28.0000
91    share last quarter  4486.000000  4486.000000  Default  27.0000  27.0000
90            share last  4560.000000  4560.000000  Default  26.0000  26.0000
..                   ...          ...          ...      ...      ...      ...
64        rating company     3.072484  2761.705672  Topic10  -4.6052   0.0502
68       rating research     3.072473  3866.496347  Topic10  -4.6052  -0.2863
39          move average     3.072471  5890.448921  Topic10  -4.6052  -0.7073
87         share company     3.072463  4917.995642  Topic10  -4.6052  -0.5269
86            sell share     3.072459  2160.434444  Topic10  -4.6052   0.2957

[424 rows x 6 columns], token_table=      Topic      Freq                Term
term                                     
0         1  0.266696  acquire additional
0         3  0.550880  acquire additional
0         4  0.003400  acquire additional
0         5  0.171968  acquire additional
0         7  0.000486  acquire additional
...     ...       ...                 ...
99        6  0.050458        target price
99        7  0.123780        target price
99        8  0.041786        target price
99        9  0.042771        target price
99       10  0.000591        target price

[866 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 8, 9, 7, 3, 6, 5, 2, 10])

when max_features = 500

In [25]:
n_top_words = 10
vectorizer = CountVectorizer(min_df = 0.1, max_df = 0.9,
                             max_features=500,
                             ngram_range=(2,4))
data_vectorized = vectorizer.fit_transform(data_lemmatized)
lda_model = LatentDirichletAllocation(n_components=10,
                                      max_iter=10,
                                      learning_method='online',   
                                      random_state=1,
                                      learning_offset=10.)
lda_output = lda_model.fit_transform(data_vectorized)
tf_feature_names = vectorizer.get_feature_names()
topics = dict()
for topic_idx, topic in enumerate(lda_model.components_):
    topics[topic_idx] = [tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
    print("Topic #%d:" % topic_idx)
    print(" | ".join([tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
print("Log Likelihood: ", lda_model.score(data_vectorized))
print("Perplexity: ", lda_model.perplexity(data_vectorized))
pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)
vis

Topic #0:
buy additional | buy additional share | own share | additional share | buy additional share last | share last | additional share last | last quarter | share last quarter | additional share last quarter
Topic #1:
value acquire | acquire additional | acquire additional share | stock value acquire additional | stock value acquire | value acquire additional | value acquire additional share | stock value | additional share | own share
Topic #2:
share company | company stock | share company stock | own share company | own share | own share company stock | sell share | average price | share company stock value | company stock value
Topic #3:
email address | receive daily | late news | enter email | daily summary | enter email address | enter email address receive | email address receive | summary late news | company marketbeat newsletter
Topic #4:
buy rating | day move | day move average | average price | move average | hold rating | move average price | price target | day move aver

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.179161 -0.022291       1        1  16.852162
8     -0.004112  0.027182       2        1  16.167955
2      0.089194 -0.233528       3        1  13.500253
0      0.106887  0.047637       4        1  10.970972
5      0.094362  0.045891       5        1   8.557851
9      0.101751  0.024176       6        1   7.561298
6     -0.175189  0.012444       7        1   7.283766
3     -0.272487  0.011923       8        1   7.009072
7      0.123717  0.061760       9        1   6.172900
1      0.115038  0.024806      10        1   5.923772, topic_info=                    Term         Freq        Total Category  logprob  loglift
4       additional share  6659.000000  6659.000000  Default  30.0000  30.0000
422          simple move  2049.000000  2049.000000  Default  29.0000  29.0000
423  simple move average  2025.000000  2025.000000  Default  28.0000  28.0000
254            own share  8213.000000  8213.000000  Default  27.0000  27.0000
118           day simple  1969.000000  1969.000000  Default  26.0000  26.0000
..                   ...          ...          ...      ...      ...      ...
404  share company stock   285.705135  3764.606199  Topic10  -4.8956   0.2478
255    own share company   274.180658  3612.367605  Topic10  -4.9368   0.2479
403        share company   313.493920  4911.249707  Topic10  -4.8028   0.0747
187          hold rating   251.329908  5826.145452  Topic10  -5.0238  -0.3171
378        research note   235.893724  4760.760720  Topic10  -5.0872  -0.1786

[467 rows x 6 columns], token_table=      Topic      Freq                      Term
term                                           
0         3  0.000962        acquire additional
0         9  0.519262        acquire additional
0        10  0.479319        acquire additional
1         3  0.000965  acquire additional share
1         9  0.518198  acquire additional share
...     ...       ...                       ...
497       6  0.089513              year company
497       7  0.001210              year company
497       8  0.001210              year company
497       9  0.061691              year company
497      10  0.048385              year company

[1633 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 9, 3, 1, 6, 10, 7, 4, 8, 2])

when max_features = 5000

In [26]:
n_top_words = 10
vectorizer = CountVectorizer(min_df = 0.1, max_df = 0.9,
                             max_features=5000,
                             ngram_range=(2,4))
data_vectorized = vectorizer.fit_transform(data_lemmatized)
lda_model = LatentDirichletAllocation(n_components=10,
                                      max_iter=10,
                                      learning_method='online',   
                                      random_state=1,
                                      learning_offset=10.)
lda_output = lda_model.fit_transform(data_vectorized)
tf_feature_names = vectorizer.get_feature_names()
topics = dict()
for topic_idx, topic in enumerate(lda_model.components_):
    topics[topic_idx] = [tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
    print("Topic #%d:" % topic_idx)
    print(" | ".join([tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
print("Log Likelihood: ", lda_model.score(data_vectorized))
print("Perplexity: ", lda_model.perplexity(data_vectorized))
pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)
vis

Topic #0:
share company | company stock | sell share | average price | own share | sell average | sell average price | own share company | price total | average price total
Topic #1:
buy rating | price target | analyst rating | news rating | email newsletter | company daily | daily email | daily email newsletter | company daily email | company daily email newsletter
Topic #2:
day move average | day move | move average price | day move average price | average price | hold rating | buy rating | research note | earning share | move average
Topic #3:
purchase additional | purchase additional share | additional share | own share | purchase additional share last | share last | additional share last quarter | share last quarter | additional share last | last quarter
Topic #4:
company stock | share company | share company stock | own share company | own share company stock | company stock worth | share company stock value | company stock value | share company stock worth | quarter own share co

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.025674  0.017819       1        1  16.461616
0      0.057551  0.128962       2        1  13.660556
8     -0.118815  0.053494       3        1  12.896260
9      0.056566  0.005807       4        1  12.590638
3      0.118622 -0.067159       5        1  11.222070
7      0.124218 -0.067340       6        1   8.432873
5      0.133392 -0.082708       7        1   8.182736
4      0.096834  0.056425       8        1   7.520694
6     -0.278314 -0.164189       9        1   5.047260
1     -0.215727  0.118890      10        1   3.985298, topic_info=                        Term         Freq        Total Category  logprob  \
728            share company  4959.000000  4959.000000  Default  30.0000   
729      share company stock  3808.000000  3808.000000  Default  29.0000   
446        own share company  3653.000000  3653.000000  Default  28.0000   
447  own share company stock  2888.000000  2888.000000  Default  27.0000   
7           additional share  6691.000000  6691.000000  Default  26.0000   
..                       ...          ...          ...      ...      ...   
86                buy rating   629.129105  7524.509872  Topic10  -3.9499   
481             price target   565.324604  5531.739690  Topic10  -4.0568   
823             target price   343.119883  5066.533043  Topic10  -4.5562   
308               give stock   305.652365  1255.519872  Topic10  -4.6718   
55             average price   297.569859  4449.444969  Topic10  -4.6986   

     loglift  
728  30.0000  
729  29.0000  
446  28.0000  
447  27.0000  
7    26.0000  
..       ...  
86    0.7410  
481   0.9417  
823   0.5302  
308   1.8097  
55    0.5177  

[501 rows x 6 columns], token_table=      Topic      Freq                    Term
term                                         
0         1  0.000480      acquire additional
0         2  0.000480      acquire additional
0         4  0.001922      acquire additional
0         7  0.996816      acquire additional
0         8  0.000480      acquire additional
...     ...       ...                     ...
883       4  0.952547       yield ex dividend
883       9  0.004716       yield ex dividend
884       3  0.040082  yield ex dividend date
884       4  0.952545  yield ex dividend date
884       9  0.004716  yield ex dividend date

[1931 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 9, 10, 4, 8, 6, 5, 7, 2])

Overall, it seems that max_features = 100 works the best.

#### modify min_df and max_df

To modify min_df and max_df, I will try [0.05,0.95],[0.1,0.9],[0.2,0.8]

when min_df and max_df = [0.05,0.95]

In [27]:
n_top_words = 10
vectorizer = CountVectorizer(min_df = 0.05, max_df = 0.95,
                             max_features=100,
                             ngram_range=(2,4))
data_vectorized = vectorizer.fit_transform(data_lemmatized)
lda_model = LatentDirichletAllocation(n_components=10,
                                      max_iter=10,
                                      learning_method='online',   
                                      random_state=1,
                                      learning_offset=10.)
lda_output = lda_model.fit_transform(data_vectorized)
tf_feature_names = vectorizer.get_feature_names()
topics = dict()
for topic_idx, topic in enumerate(lda_model.components_):
    topics[topic_idx] = [tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
    print("Topic #%d:" % topic_idx)
    print(" | ".join([tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
print("Log Likelihood: ", lda_model.score(data_vectorized))
print("Perplexity: ", lda_model.perplexity(data_vectorized))
pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)
vis

Topic #0:
company stock | share company | share company stock | own share company | own share company stock | own share | company stock value | share company stock value | stock value | company stock worth
Topic #1:
research note | rating research note | rating research | hold rating | sell rating | move average | rating hold | neutral rating | buy rating | price target
Topic #2:
purchase additional | purchase additional share | share last | additional share last | additional share last quarter | share last quarter | last quarter | purchase additional share last | own share | additional share
Topic #3:
buy rating | average price | day move | day move average | move average | move average price | day move average price | price target | hold rating | target price
Topic #4:
purchase additional | purchase additional share | share last | share last quarter | additional share last | additional share last quarter | last quarter | own share | purchase additional share last | additional share
T

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.136834 -0.074406       1        1  18.919016
3     -0.183837 -0.012634       2        1  15.096188
7      0.159140 -0.010448       3        1  14.470602
8     -0.068603 -0.036230       4        1  11.296580
6      0.046255 -0.040851       5        1   9.847935
2      0.129253  0.022533       6        1   9.566177
5     -0.183504  0.142329       7        1   9.386947
4      0.099137  0.121583       8        1   5.881769
1     -0.151291 -0.130645       9        1   5.428980
9      0.016617  0.018769      10        1   0.105805, topic_info=                    Term         Freq        Total Category  logprob  loglift
80         research note  4735.000000  4735.000000  Default  30.0000  30.0000
88   share company stock  3768.000000  3768.000000  Default  29.0000  29.0000
69  rating research note  2051.000000  2051.000000  Default  28.0000  28.0000
91    share last quarter  4486.000000  4486.000000  Default  27.0000  27.0000
90            share last  4560.000000  4560.000000  Default  26.0000  26.0000
..                   ...          ...          ...      ...      ...      ...
64        rating company     3.072484  2761.705672  Topic10  -4.6052   0.0502
68       rating research     3.072473  3866.496347  Topic10  -4.6052  -0.2863
39          move average     3.072471  5890.448921  Topic10  -4.6052  -0.7073
87         share company     3.072463  4917.995642  Topic10  -4.6052  -0.5269
86            sell share     3.072459  2160.434444  Topic10  -4.6052   0.2957

[424 rows x 6 columns], token_table=      Topic      Freq                Term
term                                     
0         1  0.266696  acquire additional
0         3  0.550880  acquire additional
0         4  0.003400  acquire additional
0         5  0.171968  acquire additional
0         7  0.000486  acquire additional
...     ...       ...                 ...
99        6  0.050458        target price
99        7  0.123780        target price
99        8  0.041786        target price
99        9  0.042771        target price
99       10  0.000591        target price

[866 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 8, 9, 7, 3, 6, 5, 2, 10])

when min_df and max_df = [0.1,0.9]

In [28]:
n_top_words = 10
vectorizer = CountVectorizer(min_df = 0.1, max_df = 0.9,
                             max_features=100,
                             ngram_range=(2,4))
data_vectorized = vectorizer.fit_transform(data_lemmatized)
lda_model = LatentDirichletAllocation(n_components=10,
                                      max_iter=10,
                                      learning_method='online',   
                                      random_state=1,
                                      learning_offset=10.)
lda_output = lda_model.fit_transform(data_vectorized)
tf_feature_names = vectorizer.get_feature_names()
topics = dict()
for topic_idx, topic in enumerate(lda_model.components_):
    topics[topic_idx] = [tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
    print("Topic #%d:" % topic_idx)
    print(" | ".join([tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
print("Log Likelihood: ", lda_model.score(data_vectorized))
print("Perplexity: ", lda_model.perplexity(data_vectorized))
pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)
vis

Topic #0:
company stock | share company | share company stock | own share company | own share company stock | own share | company stock value | share company stock value | stock value | company stock worth
Topic #1:
research note | rating research note | rating research | hold rating | sell rating | move average | rating hold | neutral rating | buy rating | price target
Topic #2:
purchase additional | purchase additional share | share last | additional share last | additional share last quarter | share last quarter | last quarter | purchase additional share last | own share | additional share
Topic #3:
buy rating | average price | day move | day move average | move average | move average price | day move average price | price target | hold rating | target price
Topic #4:
purchase additional | purchase additional share | share last | share last quarter | additional share last | additional share last quarter | last quarter | own share | purchase additional share last | additional share
T

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.136834 -0.074406       1        1  18.919016
3     -0.183837 -0.012634       2        1  15.096188
7      0.159140 -0.010448       3        1  14.470602
8     -0.068603 -0.036230       4        1  11.296580
6      0.046255 -0.040851       5        1   9.847935
2      0.129253  0.022533       6        1   9.566177
5     -0.183504  0.142329       7        1   9.386947
4      0.099137  0.121583       8        1   5.881769
1     -0.151291 -0.130645       9        1   5.428980
9      0.016617  0.018769      10        1   0.105805, topic_info=                    Term         Freq        Total Category  logprob  loglift
80         research note  4735.000000  4735.000000  Default  30.0000  30.0000
88   share company stock  3768.000000  3768.000000  Default  29.0000  29.0000
69  rating research note  2051.000000  2051.000000  Default  28.0000  28.0000
91    share last quarter  4486.000000  4486.000000  Default  27.0000  27.0000
90            share last  4560.000000  4560.000000  Default  26.0000  26.0000
..                   ...          ...          ...      ...      ...      ...
64        rating company     3.072484  2761.705672  Topic10  -4.6052   0.0502
68       rating research     3.072473  3866.496347  Topic10  -4.6052  -0.2863
39          move average     3.072471  5890.448921  Topic10  -4.6052  -0.7073
87         share company     3.072463  4917.995642  Topic10  -4.6052  -0.5269
86            sell share     3.072459  2160.434444  Topic10  -4.6052   0.2957

[424 rows x 6 columns], token_table=      Topic      Freq                Term
term                                     
0         1  0.266696  acquire additional
0         3  0.550880  acquire additional
0         4  0.003400  acquire additional
0         5  0.171968  acquire additional
0         7  0.000486  acquire additional
...     ...       ...                 ...
99        6  0.050458        target price
99        7  0.123780        target price
99        8  0.041786        target price
99        9  0.042771        target price
99       10  0.000591        target price

[866 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 8, 9, 7, 3, 6, 5, 2, 10])

when min_df and max_df = [0.15,0.85]

In [29]:
n_top_words = 10
vectorizer = CountVectorizer(min_df = 0.15, max_df = 0.85,
                             max_features=100,
                             ngram_range=(2,4))
data_vectorized = vectorizer.fit_transform(data_lemmatized)
lda_model = LatentDirichletAllocation(n_components=10,
                                      max_iter=10,
                                      learning_method='online',   
                                      random_state=1,
                                      learning_offset=10.)
lda_output = lda_model.fit_transform(data_vectorized)
tf_feature_names = vectorizer.get_feature_names()
topics = dict()
for topic_idx, topic in enumerate(lda_model.components_):
    topics[topic_idx] = [tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
    print("Topic #%d:" % topic_idx)
    print(" | ".join([tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
print("Log Likelihood: ", lda_model.score(data_vectorized))
print("Perplexity: ", lda_model.perplexity(data_vectorized))
pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)
vis

Topic #0:
company stock | own share | last quarter | share last | additional share last | share last quarter | additional share last quarter | share company | additional share | purchase additional
Topic #1:
research note | rating research note | rating research | hold rating | sell rating | neutral rating | rating hold | rating buy | buy rate | research analyst
Topic #2:
company stock | share company | simple move | simple move average | day simple move | day simple | day simple move average | price target | share company stock | sell share
Topic #3:
price target | rating share | hold rating | day move | day move average | research report | sell rating | target price | neutral rating | report may
Topic #4:
share last | share last quarter | additional share last | additional share last quarter | last quarter | own share | additional share | stock worth | provider stock | acquire additional
Topic #5:
target price | price share | target price share | rating issue | average price | rating

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.134295 -0.072136       1        1  16.953604
0     -0.179543  0.048139       2        1  15.316432
8      0.072188  0.007213       3        1  13.801469
7     -0.119756  0.084277       4        1  12.742551
3      0.198223  0.011972       5        1  11.059838
6      0.061457 -0.021957       6        1   9.553679
9     -0.206894 -0.116375       7        1   6.997714
2     -0.042283  0.103054       8        1   5.378141
1      0.188150 -0.197850       9        1   4.781172
5      0.162753  0.153664      10        1   3.415400, topic_info=                    Term         Freq        Total Category  logprob  loglift
77         research note  4768.000000  4768.000000  Default  30.0000  30.0000
98          target price  5030.000000  5030.000000  Default  29.0000  29.0000
66  rating research note  2066.000000  2066.000000  Default  28.0000  28.0000
47           price share  1885.000000  1885.000000  Default  27.0000  27.0000
65       rating research  3858.000000  3858.000000  Default  26.0000  26.0000
..                   ...          ...          ...      ...      ...      ...
57           quick ratio    78.309877  2474.189393  Topic10  -4.7986  -0.0761
93            stock sell    75.578826  2586.794208  Topic10  -4.8341  -0.1561
75            report may    69.574105  2285.155484  Topic10  -4.9169  -0.1149
33           hold rating   102.182492  5818.337112  Topic10  -4.5325  -0.6651
16         company stock   110.701271  7316.648579  Topic10  -4.4524  -0.8142

[392 rows x 6 columns], token_table=      Topic      Freq                Term
term                                     
0         1  0.548538  acquire additional
0         2  0.000974  acquire additional
0         3  0.003897  acquire additional
0         4  0.423826  acquire additional
0         6  0.022896  acquire additional
...     ...       ...                 ...
98       10  0.353835        target price
99        2  0.002598  target price share
99        6  0.002598  target price share
99        8  0.000650  target price share
99       10  0.993827  target price share

[827 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 9, 8, 4, 7, 10, 3, 2, 6])

when min_df and max_df = [0.2,0.8]

In [30]:
n_top_words = 10
vectorizer = CountVectorizer(min_df = 0.2, max_df = 0.8,
                             max_features=100,
                             ngram_range=(2,4))
data_vectorized = vectorizer.fit_transform(data_lemmatized)
lda_model = LatentDirichletAllocation(n_components=10,
                                      max_iter=10,
                                      learning_method='online',   
                                      random_state=1,
                                      learning_offset=10.)
lda_output = lda_model.fit_transform(data_vectorized)
tf_feature_names = vectorizer.get_feature_names()
topics = dict()
for topic_idx, topic in enumerate(lda_model.components_):
    topics[topic_idx] = [tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
    print("Topic #%d:" % topic_idx)
    print(" | ".join([tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
print("Log Likelihood: ", lda_model.score(data_vectorized))
print("Perplexity: ", lda_model.perplexity(data_vectorized))
pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)
vis

Topic #0:
share last | additional share last | share last quarter | additional share last quarter | last quarter | own share | additional share | purchase additional share last | stock worth | purchase additional
Topic #1:
simple move | simple move average | day simple | day simple move | day simple move average | research note | sell rating | rating research | neutral rating | price objective
Topic #2:
quarter value | earning share | target price | fourth quarter | position share | institutional investor | revenue quarter | hedge fund | first quarter | revenue quarter compare
Topic #3:
average price | move average price | day move average price | day move | day move average | research note | rating research | rating research note | sell rating | neutral rating
Topic #4:
research report | rating research report | rating research | purchase additional | purchase additional share | additional share | report may | own share | stock value | target price
Topic #5:
target price | day move | 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.189042 -0.077872       1        1  19.810575
6      0.131989  0.157038       2        1  16.754280
5     -0.205992 -0.010732       3        1  10.340767
3     -0.204573  0.005277       4        1  10.007231
1     -0.239952 -0.074777       5        1   9.763058
7      0.139330 -0.046520       6        1   8.595458
8      0.167541 -0.093200       7        1   8.400656
2      0.004779  0.001705       8        1   8.249214
4      0.021426 -0.099648       9        1   4.451712
9     -0.003590  0.238731      10        1   3.627049, topic_info=                     Term         Freq        Total Category  logprob  loglift
46           price target  5479.000000  5479.000000  Default  30.0000  30.0000
77        research report  3524.000000  3524.000000  Default  29.0000  29.0000
84    share company stock  3739.000000  3739.000000  Default  28.0000  28.0000
87     share last quarter  4542.000000  4542.000000  Default  27.0000  27.0000
3   additional share last  4542.000000  4542.000000  Default  26.0000  26.0000
..                    ...          ...          ...      ...      ...      ...
59         rating company    80.078467  2760.848132  Topic10  -4.8098  -0.2235
63        rating research   102.188429  3855.742856  Topic10  -4.5660  -0.3137
40              own share   156.725214  8265.749100  Topic10  -4.1383  -0.6486
84    share company stock    80.530364  3739.452389  Topic10  -4.8042  -0.5213
26      debt equity ratio    69.396197  2393.648612  Topic10  -4.9530  -0.2240

[382 rows x 6 columns], token_table=      Topic      Freq                      Term
term                                           
0         1  0.631204        acquire additional
0         2  0.080691        acquire additional
0         6  0.285045        acquire additional
0        10  0.002387        acquire additional
1         1  0.632355  acquire additional share
...     ...       ...                       ...
99        4  0.001384              target share
99        5  0.001384              target share
99        7  0.000692              target share
99        8  0.000692              target share
99       10  0.986205              target share

[798 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 7, 6, 4, 2, 8, 9, 3, 5, 10])

From the results shown above, it seems that min_df and max_df = [0.05,0.95] or [0.1,0.9] works the best.
Choose [0.05,0.95] for following analysis.

### run the best model

Conclude from previous analysis, the best parameters are:

max_iter = 10, max_features = 100, min_df = 0.05, max_df = 0.95.

In [36]:
# Building the tf, check sparsicity
vectorizer = CountVectorizer(max_df=0.95, min_df=0.05, max_features=100,
                             ngram_range=(2,4))
data_vectorized = vectorizer.fit_transform(data_lemmatized)
data_dense = data_vectorized.todense()
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  49.55900996076064 %


In [37]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=10,
                                      max_iter=10,
                                      learning_method='online',   
                                      random_state=1,
                                      learning_offset=10.)
lda_output = lda_model.fit_transform(data_vectorized)

In [32]:
# Print out topics
tf_feature_names = vectorizer.get_feature_names()

topics = dict()
for topic_idx, topic in enumerate(lda_model.components_):
    topics[topic_idx] = [tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
    print("Topic #%d:" % topic_idx)
    print(" | ".join([tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

Topic #0:
sell share | additional share last | share company | additional share last quarter | hedge fund | move average day | additional share | price target | stock sell rating | price objective
Topic #1:
share last | share last quarter | day move average | day move average price | day simple | ratio quick | research report | rating buy | move average | own share
Topic #2:
purchase additional share last | debt equity | stock value | equity ratio | neutral rating | fourth quarter | research analyst | first quarter | earning share | research note
Topic #3:
average day | last year | day move | company stock worth | current ratio | ratio quick | rating buy | rating company | research report | move average
Topic #4:
ratio quick ratio | rating hold | rating buy | price objective | price share | additional share | ratio current | move average day | stock sell | stock value
Topic #5:
stock value | company stock worth | current ratio | own share company stock | rating report | research report

In [38]:
# Visualization
import pyLDAvis
pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.136834 -0.074406       1        1  18.919016
3     -0.183837 -0.012634       2        1  15.096188
7      0.159140 -0.010448       3        1  14.470602
8     -0.068603 -0.036230       4        1  11.296580
6      0.046255 -0.040851       5        1   9.847935
2      0.129253  0.022533       6        1   9.566177
5     -0.183504  0.142329       7        1   9.386947
4      0.099137  0.121583       8        1   5.881769
1     -0.151291 -0.130645       9        1   5.428980
9      0.016617  0.018769      10        1   0.105805, topic_info=                    Term         Freq        Total Category  logprob  loglift
80         research note  4735.000000  4735.000000  Default  30.0000  30.0000
88   share company stock  3768.000000  3768.000000  Default  29.0000  29.0000
69  rating research note  2051.000000  2051.000000  Default  28.0000  28.0000
91    share last quarter  4486.000000  4486.000000  Default  27.0000  27.0000
90            share last  4560.000000  4560.000000  Default  26.0000  26.0000
..                   ...          ...          ...      ...      ...      ...
64        rating company     3.072484  2761.705672  Topic10  -4.6052   0.0502
68       rating research     3.072473  3866.496347  Topic10  -4.6052  -0.2863
39          move average     3.072471  5890.448921  Topic10  -4.6052  -0.7073
87         share company     3.072463  4917.995642  Topic10  -4.6052  -0.5269
86            sell share     3.072459  2160.434444  Topic10  -4.6052   0.2957

[424 rows x 6 columns], token_table=      Topic      Freq                Term
term                                     
0         1  0.266696  acquire additional
0         3  0.550880  acquire additional
0         4  0.003400  acquire additional
0         5  0.171968  acquire additional
0         7  0.000486  acquire additional
...     ...       ...                 ...
99        6  0.050458        target price
99        7  0.123780        target price
99        8  0.041786        target price
99        9  0.042771        target price
99       10  0.000591        target price

[866 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 8, 9, 7, 3, 6, 5, 2, 10])

In [39]:
print("Log Likelihood: ", lda_model.score(data_vectorized))
print("Perplexity: ", lda_model.perplexity(data_vectorized))

Log Likelihood:  -1270638.0628126084
Perplexity:  79.49438354849791


# Applying LDA model (50 pts)

Pick 10 random articles,  run them through the trained LDA model, and show which topic cluster each falls into. Print out the cluster IDs and keywords.

#### pick 10 random article and print out cluster ID and keywords

In [52]:
# pick 10 random ariticles
import random
indices = []
for i in range(10):
    indices.append(int(random.random()*len(feeds)))
print(indices)

[303, 649, 246, 3285, 1556, 3058, 3231, 2254, 2879, 364]


In [66]:
# calculate results for all articles

import pandas as pd
import numpy as np

lda_output = lda_model.fit_transform(data_vectorized)

topicnames = ["Topic" + str(i) for i in topics]
docnames = ["Doc" + str(i) for i in range(len(feeds))]
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

In [79]:
# dominant topic id & keywords for picked ariticles
for index in indices:
    print("For doc #", index)
    print("Dominant topic : Topic #", dominant_topic[index])
    print("Keywords", topics[dominant_topic[index]])

For doc # 303
Dominant topic : Topic # 2
Keywords ['purchase additional share last', 'debt equity', 'stock value', 'equity ratio', 'neutral rating', 'fourth quarter', 'research analyst', 'first quarter', 'earning share', 'research note']
For doc # 649
Dominant topic : Topic # 2
Keywords ['purchase additional share last', 'debt equity', 'stock value', 'equity ratio', 'neutral rating', 'fourth quarter', 'research analyst', 'first quarter', 'earning share', 'research note']
For doc # 246
Dominant topic : Topic # 7
Keywords ['move average day', 'additional share', 'share company stock', 'also recently', 'stock sell rating', 'purchase additional', 'purchase additional share', 'stock sell', 'acquire additional', 'acquire additional share']
For doc # 3285
Dominant topic : Topic # 0
Keywords ['sell share', 'additional share last', 'share company', 'additional share last quarter', 'hedge fund', 'move average day', 'additional share', 'price target', 'stock sell rating', 'price objective']
For d

#### FYI, other analysis

In [81]:
# FYI, results for all ariticles
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,dominant_topic
Doc0,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.700000,0.030000,0.030000,0.030000,6
Doc1,0.340000,0.050000,0.000000,0.180000,0.000000,0.000000,0.000000,0.430000,0.000000,0.000000,7
Doc2,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0
Doc3,0.140000,0.310000,0.000000,0.230000,0.000000,0.000000,0.000000,0.000000,0.320000,0.000000,8
Doc4,0.230000,0.000000,0.000000,0.000000,0.000000,0.470000,0.000000,0.000000,0.290000,0.000000,5
Doc5,0.480000,0.030000,0.130000,0.200000,0.000000,0.000000,0.090000,0.000000,0.070000,0.000000,0
Doc6,0.000000,0.060000,0.740000,0.000000,0.000000,0.000000,0.190000,0.000000,0.000000,0.000000,2
Doc7,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0
Doc8,0.000000,0.000000,0.000000,0.980000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3
Doc9,0.000000,0.000000,0.000000,0.980000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3


In [87]:
# FYI, review topics distribution across documents
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Numbers of Documents")
df_topic_distribution.columns = ['Topic #', 'Numbers of Documents']
df_topic_distribution.sort_values('Topic #', inplace=True)
df_topic_distribution

,Topic #,Numbers of Documents
1,0,665
8,1,24
5,2,308
0,3,687
7,4,180
4,5,344
6,6,269
2,7,438
3,8,398
